## Download Libraries

In [ ]:
! pip install sentence-splitter

     |████████████████████████████████| 51kB 7.2MB/s 


In [ ]:
! pip install transformers

     |████████████████████████████████| 2.3MB 24.6MB/s 
     |████████████████████████████████| 3.3MB 40.7MB/s 
     |████████████████████████████████| 901kB 51.1MB/s 


In [ ]:
! pip install SentencePiece

     |████████████████████████████████| 1.2MB 28.8MB/s 


## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

datapath = '/content/drive/My Drive/all data/'

Mounted at /content/drive


In [ ]:
import os
import pandas as pd

alldocs= []
allheadlines= []
for filename in os.listdir(datapath):
  if filename.startswith(r"all"):
    df = pd.read_csv(datapath + filename, encoding = 'utf-8', delimiter = ',')
    headlines = df['headline'].tolist()
    docs = df['content'].tolist()
    print("Number of documents: ", len(docs))
    alldocs.extend(docs)
    allheadlines.extend(headlines)

Number of documents:  8433


In [ ]:
print(alldocs[0])
print(allheadlines[0])

['&lt;p&gt;Younger people continue to bear the brunt of the jobs crisis amid widespread cuts in sectors such as hospitality, official figures show. &lt;p&gt;&lt;p&gt;In the year to March, 811,000 payroll jobs were lost in the UK, with under-35s accounting for 80% of these cuts. &lt;p&gt;&lt;p&gt;The data also showed the unemployment rate dipped to 4.9% in the three months to February - down from 5% previously. &lt;p&gt;&lt;p&gt;This was despite most of the UK being under strict lockdown rules for at least some of the period. &lt;p&gt;&lt;p&gt;The Office for National Statistics (ONS) said the jobs market "remains subdued", with five million people employed but still on furlough. &lt;p&gt;&lt;p&gt;Head of economics at the British Chambers of Commerce, Surenâ\x80¯Thiru, said:â\x80¯ "Unemployment remains on course to peak towards the end of 2021, once the furlough scheme expires and those who stopped job hunting during the pandemic look to return to the workforce as restrictions ease. &lt;

In [ ]:
import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

## Sentence Splitting & Add Paragraph Tags

In [ ]:
context = alldocs[0]
print(context)
context =context.split('&lt;p&gt;')
while '[\'' in context: context.remove('[\'')
while '\']' in context: context.remove('\']')
while '' in context: context.remove('')

from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list=[]
for sent in context:
  sentence_list.append(splitter.split(sent))


['&lt;p&gt;Younger people continue to bear the brunt of the jobs crisis amid widespread cuts in sectors such as hospitality, official figures show. &lt;p&gt;&lt;p&gt;In the year to March, 811,000 payroll jobs were lost in the UK, with under-35s accounting for 80% of these cuts. &lt;p&gt;&lt;p&gt;The data also showed the unemployment rate dipped to 4.9% in the three months to February - down from 5% previously. &lt;p&gt;&lt;p&gt;This was despite most of the UK being under strict lockdown rules for at least some of the period. &lt;p&gt;&lt;p&gt;The Office for National Statistics (ONS) said the jobs market "remains subdued", with five million people employed but still on furlough. &lt;p&gt;&lt;p&gt;Head of economics at the British Chambers of Commerce, Surenâ\x80¯Thiru, said:â\x80¯ "Unemployment remains on course to peak towards the end of 2021, once the furlough scheme expires and those who stopped job hunting during the pandemic look to return to the workforce as restrictions ease. &lt;

## Generate Paraphrased Content

In [ ]:
length = len(alldocs)
print(length)

contexts = []
for i in range(0,length):
  # Paragraph of text
  context = alldocs[i]
  originalcontext = context

  context = context.replace('&lt;p&gt;','')
  # Takes the input paragraph and splits it into a list of sentences
  from sentence_splitter import SentenceSplitter, split_text_into_sentences

  splitter = SentenceSplitter(language='en')

  sentence_list = splitter.split(context)
  #print(sentence_list[2])

  # Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
  paraphrase = []

  for i in sentence_list:
    a = get_response(i,1)
    paraphrase.append(a)

  #paraphrase

  paraphrase2 = [' '.join(x) for x in paraphrase]
  #paraphrase2

  # Combines the above list into a paragraph
  paraphrase3 = [' '.join(x for x in paraphrase2) ]
  paraphrased_text = str(paraphrase3).strip('[]').strip("'")
  #paraphrased_text
  
    #print(paraphrased_text)
  # Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)
  
  originalcontext =originalcontext.split('&lt;p&gt;')
  while '[\'' in originalcontext: originalcontext.remove('[\'')
  while '\']' in originalcontext: originalcontext.remove('\']')
  while '' in originalcontext: originalcontext.remove('')


  #print(len(originalcontext))
  lengths = []
  indexes = []
  totalsens = 0
  for e in originalcontext:
    sents = splitter.split(e)
    if len(sents) > 1:
      lengths.append(len(sents))
      indexes.append(originalcontext.index(e)+ totalsens)
      totalsens = totalsens + len(sents) - 1
  #print(lengths)
  #print(indexes)

  paraphrased_text = paraphrased_text.replace('bbc', 'hello')      
  paraphrased_text = splitter.split(paraphrased_text)

  #print(len(originalcontext))
  #print(len(paraphrased_text))

  while '[\'' in paraphrased_text: paraphrased_text.remove('[\'')
  while '\']' in paraphrased_text: paraphrased_text.remove('\']')
  while '' in paraphrased_text: paraphrased_text.remove('')
  #count = 0

  #print(lengths)
  #print(indexes)
  #print(originalcontext)
  #print(paraphrased_text)
  #print(len(paraphrased_text))
 

  count = 0
  
  paraph =""
  if lengths == []:
    paraph += '&lt;p&gt;'
    for i in range(0,len(paraphrased_text)):
        paraph += paraphrased_text[i]
    paraph += '&lt;p&gt;'

  #elif len(paraphrased_text) >= lengths[count]:
  else:
    for i in range(0,len(paraphrased_text)):
      i = i + count
      if i < len(paraphrased_text):
        if i in indexes:
          paraph += '&lt;p&gt;'
          for j in range(0, lengths[indexes.index(i)]):
              if i+j < (len(paraphrased_text)):
                paraph += paraphrased_text[i+j]
                count = count + j
          paraph += '&lt;p&gt;'
        else:
            paraph += '&lt;p&gt;'
            paraph += paraphrased_text[i]
            paraph += '&lt;p&gt;'
    
  paraph = paraph.replace('.','')

  contexts.append(paraph)
  print(paraph)
  #print(paraphrased_text)
  

8433


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3260: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


&lt;p&gt;China is making it easier for foreigners to enterThey need to have received a vaccine made in China&lt;p&gt;&lt;p&gt;Over the past week, at least 23 Chinese embassies around the world issued new visa policies with this condition, including in the United States and the United Kingdom, where Chinese vaccines aren\'t available&lt;p&gt;&lt;p&gt;China\'s foreign ministry says the move is about kick-starting international travel in an "orderly fashion"&lt;p&gt;&lt;p&gt;But experts have raised concerns over China\'s decision to prioritize domestic vaccines over those approved by the World Health Organization&lt;p&gt;&lt;p&gt;It sets a dangerous precedent which could leave the world in vaccine-based silos if other nations adopt it&lt;p&gt;&lt;p&gt;Nicholas Thomas, an associate professor in health security at the City University of Hong Kong, said that it\'s very much at the end of vaccine diplomacy&lt;p&gt;&lt;p&gt;The timing of China\'s new visa rules is notableAfter the Quad, a part